<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Invoice_Processing_with_Amazon_Textract_and_and_GPT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3  openai -q 

In [ ]:
import openai
import boto3
openai.api_key = "sk-HUu"

In [3]:
client = boto3.client('textract',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

In [ ]:
filename = "/content/invoice.png"
from PIL import Image, ImageDraw
image=Image.open(filename)
image

In [ ]:
with open(filename, 'rb') as image:
    img = bytearray(image.read())


response = client.detect_document_text(
    Document={'Bytes': img}
)
response

In [ ]:
response

In [7]:
ocr_text = ""
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        # print (item["Text"])
        ocr_text = ocr_text + " "+item["Text"]
ocr_text                

' 06/10/2021 K Company INVO-005 Name Sample Invoice Billing Information Shipping Information Company Name Name ABC Company John Smith Sam K. Smith Address Address 111 Pine Street, Suite 1815 111 Pine Street, Suite 1815 San Francisco, CA, 94111 San Francisco, CA, 94111 Phone Number (123) 123-1232 Email John@example.com Description Quantity Unit Price Total Product/Service 1 Sink 2 100 $200 Product/Service 2 Nest Smart Filter 1 150 $150 Product/Service 3 Labor Fee 1 50 $50 Product/Service 4 Service Fee 1 25 $25 Total: $425 1'

In [8]:
prompt= f"""Extract entities and thier values as a key-value pair from the provided OCR text and seperate them by a new line.
Text:{ocr_text} 
Entities:"""
print(prompt)

Extract entities and thier values as a key-value pair from the provided OCR text and seperate them by a new line.
Text: 06/10/2021 K Company INVO-005 Name Sample Invoice Billing Information Shipping Information Company Name Name ABC Company John Smith Sam K. Smith Address Address 111 Pine Street, Suite 1815 111 Pine Street, Suite 1815 San Francisco, CA, 94111 San Francisco, CA, 94111 Phone Number (123) 123-1232 Email John@example.com Description Quantity Unit Price Total Product/Service 1 Sink 2 100 $200 Product/Service 2 Nest Smart Filter 1 150 $150 Product/Service 3 Labor Fee 1 50 $50 Product/Service 4 Service Fee 1 25 $25 Total: $425 1 
Entities:


In [9]:
#Get The Response
gpt3_response= openai.Completion.create(
model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(gpt3_response["choices"][0]["text"])

 
Date: 06/10/2021 
Company Name: K Company 
Invoice Number: INVO-005 
Name: Sample Invoice 
Billing Information Company Name: ABC Company 
Billing Information Name: John Smith 
Shipping Information Name: Sam K. Smith 
Billing Information Address: 111 Pine Street, Suite 1815 
Shipping Information Address: 111 Pine Street, Suite 1815 
Billing Information City: San Francisco 
Shipping Information City: San Francisco 
Billing Information State: CA 
Shipping Information State: CA 
Billing Information Zip Code: 94111 
Shipping Information Zip Code: 94111 
Billing Information Phone Number: (123) 123-1232 
Billing Information Email: John@example.com 
Product/Service 1: Sink 
Product/Service 1 Quantity: 2 
Product/Service 1 Unit Price: 100 
Product/Service 1 Total: $200 
Product/Service 2: Nest Smart Filter 
Product/Service 2 Quantity: 1 
Product/Service 2 Unit Price: 150 
Product/Service 2 Total: $150 
Product/Service 3: Labor Fee 
Product/Service 3 Quantity: 1 
Product/Service 3 Unit Price: 

In [10]:
system_role="Extract entities and thier values as a key-value pair from the provided OCR text and seperate them by a new line."
gpt4_response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role":"system","content":system_role},
        {"role":"user","content":ocr_text} #pass the OCR Text obtained from Amazon Textract
    ]
)
print(gpt4_response["choices"][0]["message"]["content"])

Date: 06/10/2021
Invoice Number: INVO-005
CompanyName: K Company
Name: Sample Invoice
Billing Name: ABC Company
Billing Address: 111 Pine Street, Suite 1815, San Francisco, CA, 94111
Billing Phone Number: (123) 123-1232
Shipping Name: John Smith Sam K. Smith
Shipping Address: 111 Pine Street, Suite 1815, San Francisco, CA, 94111
Email: John@example.com
Product/Service 1: Sink
Quantity 1: 2
Unit Price 1: 100
Total 1: $200
Product/Service 2: Nest Smart Filter
Quantity 2: 1
Unit Price 2: 150
Total 2: $150
Product/Service 3: Labor Fee
Quantity 3: 1
Unit Price 3: 50
Total 3: $50
Product/Service 4: Service Fee
Quantity 4: 1
Unit Price 4: 25
Total 4: $25
Invoice Total: $425


In [11]:
system_role="Extract company name and invoice number as a key-value pair from the provided OCR text and seperate them by a new line."
gpt4_response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role":"system","content":system_role},
        {"role":"user","content":ocr_text} #pass the OCR Text obtained from Amazon Textract
    ]
)
print(gpt4_response["choices"][0]["message"]["content"])

Company Name: ABC Company
Invoice Number: INVO-005


In [12]:
system_role="You are AI Asistsnt who helps answers questions based on providec ocr text"
gpt4_response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role":"system","content":system_role},
        {"role":"user","content":"OCR Text: "+ocr_text},
        {"role":"user","content":"What is total order?"} #pass the OCR Text obtained from Amazon Textract
    ]
)
gpt4_response["choices"][0]["message"]["content"]

'The total order amount is $425.'

In [13]:
system_role="You are AI Asistsnt who helps answers questions based on providec ocr text"
gpt4_response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role":"system","content":system_role},
        {"role":"user","content":"OCR Text: "+ocr_text},
        {"role":"user","content":"How many units sold for service 1"} #pass the OCR Text obtained from Amazon Textract
    ]
)
gpt4_response["choices"][0]["message"]["content"]

'Service 1, which is for the Sink, sold 2 units.'